In [ ]:
import numpy as np 
import pandas as pd
import os


In [ ]:

movies = pd.read_csv(os.path.join("Resources\\tmdb_5000_movies.csv"))
credits = pd.read_csv(os.path.join("Resources\\tmdb_5000_credits.csv"))



In [ ]:

movies.head()

In [ ]:
movies = movies.merge(credits, on="title")

In [ ]:
movies.head(1)

In [ ]:
# genres
# id
# keywords
# title
# overview
# cast
movies= movies[["movie_id","title","overview","genres","keywords",'cast',"crew"]]

# need to make tag from overview, genres, kerword, cast, make one tag paragraph for model 

In [ ]:
movies.head()

In [ ]:
movies.isnull().sum()

### dropna drops all null value rows

In [ ]:
movies.dropna(inplace=True)

### to check if any row is duplicated

In [ ]:
movies.duplicated().sum()

In [ ]:
movies.iloc[0].genres

In [ ]:
import ast

def convert(obj):
    L = []
    for i in ast.literal_eval(obj):
        L.append(i["name"])
    return L

In [ ]:
# apply works just like JS foreach(), we give in function in args, and it iterate its each values

In [ ]:
movies["genres"]=movies["genres"].apply(convert)

In [ ]:
movies["keywords"]=movies["keywords"].apply(convert)

In [ ]:
import ast

def convert3(obj):
    counter =0
    L = []
    for i in ast.literal_eval(obj):
        if counter!=3:
            L.append(i["name"])
            counter+=1
        else:
            break
    return L

In [ ]:
movies["cast"]=movies["cast"].apply(convert3)

In [ ]:
movies["cast"]

In [ ]:
import ast

def convertForCrew(obj):
    L = []
    for i in ast.literal_eval(obj):
        if i["job"]=="Director":
            L.append(i["name"])
            break

    return L

In [ ]:
movies['crew']=movies["crew"].apply(convertForCrew)

## to make overview column list from string

In [ ]:
movies["overview"]=movies["overview"].apply(lambda x:x.split())

### single line loop, firstly write single loop line, then write loop

In [ ]:
movies["genres"]=movies["genres"].apply(lambda x:[i.replace(" ","") for i in x])
movies["keywords"]=movies["keywords"].apply(lambda x:[i.replace(" ","") for i in x])
movies["cast"]=movies["cast"].apply(lambda x:[i.replace(" ","") for i in x])
movies["crew"]=movies["crew"].apply(lambda x:[i.replace(" ","") for i in x])

movies.head()





In [ ]:
movies['tags']= movies['overview']+movies["genres"]+movies["keywords"]+movies["cast"]+movies["crew"]

In [ ]:
movies.head()
new_df = movies[["movie_id","title","tags"]]


### converting lst to string using lambda

In [ ]:
new_df["tags"]=new_df["tags"].apply(lambda x:" ".join(x))

In [ ]:
new_df.head()

### lowering chars in tags

In [ ]:
new_df["tags"]=new_df["tags"].apply(lambda x:x.lower())

In [ ]:
new_df["tags"][0]

In [ ]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [ ]:
def stem(text):
    y = []
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)

In [ ]:
new_df['tags'] = new_df['tags'].apply(stem)

In [ ]:
new_df['tags'][0]

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')

In [ ]:
vectors = cv.fit_transform(new_df['tags']).toarray()

In [ ]:
cv.get_feature_names()

In [ ]:
stem('in the 22nd century, a paraplegic marine is dispatched to the moon pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization. action adventure fantasy sciencefiction cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d samworthington zoesaldana sigourneyweaver jamescameron')

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
simalarity=cosine_similarity(vectors)

In [ ]:
sorted(list(enumerate(simalarity[0])),reverse=True,key=lambda x:x[1])[1:6]

In [ ]:
def recommend(movie):
    movie_index = new_df[new_df['title']==movie].index[0]
    distances = simalarity[movie_index]
    movies_list = sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:6]
    
    for i in movies_list:
        print(new_df.iloc[i[0]].title)
        print(i[0])



In [ ]:
recommend('Avatar')